<a href="https://colab.research.google.com/github/noemotion85/FreeTestRepository/blob/main/Chap01_Abstract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요
## 일단 해보기
*colab은 당장 딥러닝 관련 요소를 설치할 필요는 없다.

In [2]:
# CLICK ME
from fastai.vision.all import * # fastai 내 vision 패키지

path = untar_data(URLs.PETS)/'images' # untar_data(): 압축 해제;
#tar은 리눅스(유닉스) 체계에서 흔히 사용하는 압축방식

def is_cat(x): return x[0].isupper()
# 데이터로더(파이토치는 torch.utils.data.dataloader)
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224)
    )

learn = cnn_learner(dls, resnet34, metrics = error_rate)
# UserWarning: `cnn_learner` has been renamed to `vision_learner`
# -- please update your code

learn.fine_tune(1)

/usr/local/lib/python3.7/dist-packages/fastai/vision/learner.py:288: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627

  0%|          | 0.00/83.3M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,0.172359,0.007226,0.002030,32:48


epoch,train_loss,valid_loss,error_rate,time
0,0.063695,0.013353,0.004736,48:14


##### fast.ai의 데이터셋 컬렉션
URLs.PETS: https://course.fastai./datasets

코드가 실제로 작동하는지 확인해보자.

In [6]:
import ipywidgets as widgets

uploader = widgets.FileUpload()
display(uploader)

FileUpload(value={}, description='Upload')

이제 업로드한 파일을 모델로 전달 할 수 있다.
##### 비고: 업로드 버튼 생성
참고
https://github.com/jupyter-widgets/ipywidgets/issues/3452


In [ ]:
img = PILImage.create(uploader.data[0])
is_cat,_,probs = learn.predict(img)

print(f"Is this a cat?: {is_cat}.")
print(f"Probability it's a cat: {probs[1].item():.6f}")

첫 번째 분류 모델 완성!

## Now, 코드 분석!

1. path = untar_data(URLs.PETS)/'images' 는
##### fast.ai의 데이터셋 컬렉션에서 데이터 셋을 (다운로드 한 적 없으면) 다운로드한다.
##### * PETS 데이터 셋은 37개 품종으로 구성된 개와 고양이 사진을 7,390장 포함
##### ** 이미지 양식은 예를 들어, great_pyrenees_173.jpg

2. def is_cat(x): return x[0].isupper() 는
##### 데이터셋에 포함된 "파일명의 규칙"에 따라 "고양이 레이블'을 지정"하는 is_cat함수를 정의한다.
##### * 문자열의 첫 번째 문자가 대문자면 True(고양이)를 반환한다.

3. dls = ImageDataLoaders.from_name_func( path, get_image_files(path), valid_pct=0.2, seed=42, label_func=is_cat, item_tfms=Resize(224) ) 에서
### from_name_func() 는
##### 파일명에서 레이블을 가져오는 방법
### 인자 path 는
##### fast.ai 데이터셋 컬렉션( https://course.fastai./datasets )
### 인자 get_image_files(path) 는
##### path에서 image파일들을 얻어 오는(로드 하는) 함수
### 인자 valid_pct = 0.2 는
##### 데이터의 20%를 "검증용 데이터셋(validation set)"으로 설정
### 인자 seed = 42 는
##### 코드를 실행할 때마다 "동일한 임의성"으로 검증용 데이터셋을 구성하도록 시드값을 고정
### 인자 label_func = is_cat 은
##### (커스텀?) 레이블 지정; is_cat함수를 쓰도록 지시
### 인자 item_tfms = Resize(224) 는
##### 필요한 Transform(변형) 목록을 지정한다. Transform은 학습 중 자동으로 적용되는 코드를 포함한다. Transform은 파이썬 함수를 만드는 수준으로 간단히 정의가 가능하다. 각각 "item_tfms" 와 "batch_tfms" 라는 '인자'로 '제어'된다.
##### * item_tfms 는 개별 데이터 변형, batch_items 는 배치 변형
##### ** Resize(224) 를 통해서 224 x 224 픽셀 사이즈로 변형한다.
##### *** 224 픽셀은 "영상 처리 분야에서 표준적인 크기"이다.

4. learn = cnn_learner(dls, resnet34, metrics = error_rate)
### cnn_learner() 는
##### vision_learner() 로 바뀌었다. !! 주의 !!
##### * 이미지(영상)를 처리하기 좋은 신경망 = CNN